## Background Removal with Robust PCA

In [4]:
# imports
import moviepy.editor as mpe
from IPython.display import display
from glob import glob

In [5]:
import sys, os
import numpy as np
import scipy

In [6]:
%matplotlib inline
import matplotlib.pyplot as plt

In [7]:
# MAX_ITERS = 10
TOL = 1.0e-8

In [10]:
import requests
    
c = 0
def download_file(url,local_filename):
    r = requests.get(url, stream=True)
    with open(local_filename, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024): 
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                #f.flush() commented by recommendation from J.F.Sebastian
    return local_filename

In [12]:
download_file('http://bmc.iut-auvergne.com/wp-content/ressources/videos/evaluation/Video_003.zip','Video_003.zip')

ConnectionError: HTTPConnectionPool(host='bmc.iut-auvergne.com', port=80): Max retries exceeded with url: /wp-content/ressources/videos/evaluation/Video_003.zip (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x1a0e7b5860>: Failed to establish a new connection: [Errno 60] Operation timed out',))